In [ ]:
import zabbix_db as zb
import pandas as pd
from adtk.detector import LevelShiftAD
import matplotlib.pyplot as plt

In [ ]:
db_config = {
    'db_type': 'mysql',  # or 'postgresql'
    'host': '140.238.230.93',
    'port': 3306,  # 5432 for PostgreSQL
    'database': 'zabbix',
    'user': 'kartik',
    'password': 'Kartik@24082003'
}

# Sample hostname to query
hostname = 'Zabbix server'
metric_name = 'CPU utilization'  # Example metric name
# metric_name = 'Host name of Zabbix agent running'


with zb.ZabbixDB(**db_config) as zbx:
    result = zbx.get_metric_data(
        hostname=hostname,
        metric_name=metric_name,
        time_from=1746793913,  # Example start time (Unix timestamp)
        time_to=1749549165,  # Example end time (Unix timestamp)
          # Example statistical measure
    )
    print(result)


    # result = zbx.get_host_by_metric(metric_name=metric_name,time_from=1749032410, time_to=1749118810, statistical_measure='max')
    # # result = zbx.get_item_detail(item_name='CPU nice time', hostname=hostname)
    # print(result)

In [ ]:
data = pd.DataFrame(result['data'], columns=['clock', 'value'])
print(data)
data1 = data

In [ ]:
# # Convert 'clock' from Unix timestamp to datetime and set as index
# data['timestamp'] = pd.to_datetime(data['clock'], unit='s')
# data_series = data.set_index('timestamp')['value']

# # Initialize the LevelShiftAD detector
# model = LevelShiftAD(c=1.5, side='both', window=5)

# # Detect anomalies
# anomalies = model.fit_detect(data_series)

# # Extract timestamps and values where anomalies are found
# anomaly_timestamps = anomalies[anomalies == 1].index
# anomaly_values = data_series.loc[anomaly_timestamps]

# # Create a DataFrame with anomalies
# anomaly_df = pd.DataFrame({'timestamp': anomaly_timestamps, 'value': anomaly_values})

# # Save anomalies to CSV
# anomaly_df.to_csv('anomalies.csv', index=False)
# print("Anomalies saved to anomalies.csv")

# # Visualize the data with anomalies
# plt.figure(figsize=(12, 6))
# plt.plot(data_series.index, data_series, label='Data', color='#36A2EB')
# plt.scatter(anomaly_timestamps, anomaly_values, color='#FF6384', label='Anomalies')
# plt.xlabel('Timestamp')
# plt.ylabel('Value (e.g., CPU Utilization)')
# plt.title('Anomaly Detection in IT Monitoring Data')
# plt.legend()
# plt.show()

In [ ]:
df = data

In [ ]:
import pandas as pd
from statsmodels.tsa.seasonal import STL
import matplotlib.pyplot as plt

# Ensure df is sorted by index (which is already datetime 'clock')
df = df.sort_index()

# Perform STL decomposition
stl = STL(
    df['value'],
    period=60,      # Hourly seasonality (60 minutes)
    seasonal=7,     # Seasonal smoother length
    trend=121,      # Trend smoother length
    robust=True
)
decomposition = stl.fit()

# Extract residual and calculate standard deviation
residual = decomposition.resid
std_resid = residual.std()

# Set anomaly threshold (3 deviations)
anomaly_threshold = 5 * std_resid

# Find anomalies where |residual| > threshold
anomaly_indices = residual[abs(residual) > anomaly_threshold].index

# Create output DataFrame with 'clock' (Unix timestamps) and 'value'
anomalies = pd.DataFrame({
    'clock': (anomaly_indices.astype(int) // 10**9).astype(int),
    'value': df['value'].loc[anomaly_indices]
}, index=anomaly_indices)

# Reset index to remove 'clock' as an index, keeping it only as a column
anomalies = anomalies.reset_index(drop=True)

# Sort by 'clock' for clarity
anomalies = anomalies.sort_values('clock')

# Print the anomalies DataFrame
print(anomalies)

# Visualize results
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['value'], label='Data', color='#36A2EB')
plt.scatter(anomalies['clock'].apply(lambda x: pd.Timestamp(x, unit='s')), anomalies['value'], color='#FF6384', label='Anomalies')
plt.xlabel('Timestamp')
plt.ylabel('Value (e.g., CPU Utilization)')
plt.title('Anomaly Detection with STL (trendstl)')
plt.legend()
plt.show()

In [ ]:
df=data1
df.head()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from statsmodels.robust import mad

# --- Step 1: Preprocess Full Data ---
# Assume df has datetime index and 'value' column

df = df.sort_index()
df = df.asfreq('1T')  # 1-minute data; adjust if needed
df['value'] = df['value'].interpolate(method='time')

# --- Step 2: STL Decomposition on Full Dataset ---
period = 1440        # 1 day (for 1-min data)
seasonal = 61
trend = 1921         # Must be > period and odd

stl = STL(df['value'], period=period, seasonal=seasonal, trend=trend, robust=True)
result = stl.fit()

resid = result.resid
threshold = 5 * mad(resid)

# --- Step 3: Get Last 1-Hour Residuals ---
last_hour_start = df.index.max() - pd.Timedelta(hours=1)
last_hour_resid = resid[last_hour_start:]

# --- Step 4: Pattern Check (Smart Suppression of Common Behavior) ---
def is_repeating_pattern(ts_index, df, tolerance=0.1):
    """
    Checks whether a similar value occurred often historically around the same time of day.
    Suppresses anomaly if it is part of a repeating pattern.
    """
    matches = 0
    current_value = df.loc[ts_index, 'value']
    time_of_day = ts_index.time()
    
    # Create a time window of +/- 3 minutes around that time of day for historical days
    for day_offset in range(1, 31):  # last 30 days
        try:
            compare_time = ts_index - pd.Timedelta(days=day_offset)
            compare_window = df.between_time(
                (pd.Timestamp.combine(compare_time.date(), time_of_day) - pd.Timedelta(minutes=3)).time(),
                (pd.Timestamp.combine(compare_time.date(), time_of_day) + pd.Timedelta(minutes=3)).time()
            )

            if not compare_window.empty:
                if np.any(abs(compare_window['value'] - current_value) < tolerance * current_value):
                    matches += 1
        except:
            continue

    return matches >= 2  # suppress if value occurs >=2 times historically

# --- Step 5: Identify True Anomalies (Filter Repeating Patterns) ---
anomalies = []
for ts in last_hour_resid.index:
    if abs(last_hour_resid.loc[ts]) > threshold:
        if not is_repeating_pattern(ts, df):
            anomalies.append((ts, df.loc[ts, 'value']))

# --- Step 6: Format Anomalies as DataFrame ---
anomaly_df = pd.DataFrame(anomalies, columns=['timestamp', 'value'])
anomaly_df['clock'] = anomaly_df['timestamp'].astype(np.int64) // 10**9
anomaly_df = anomaly_df[['clock', 'value']]

print("\nDetected Anomalies (Last 1 Hour, Pattern-Filtered):")
print(anomaly_df)

# --- Step 7: Plot Full Data + Anomalies ---
plt.figure(figsize=(14, 6))
plt.plot(df.index, df['value'], label='Full Data', color='#36A2EB')
plt.scatter(
    anomaly_df['clock'].apply(lambda x: pd.to_datetime(x, unit='s')),
    anomaly_df['value'],
    color='#FF6384', label='Filtered Anomalies'
)
plt.axvspan(last_hour_start, df.index.max(), color='gray', alpha=0.1, label='Last Hour')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Full STL Decomposition + Filtered Anomaly Detection (Last 1 Hour)')
plt.legend()
plt.tight_layout()
plt.show()

# --- Step 8: STL Components Plot (Optional) ---
result.plot()
plt.suptitle('STL Decomposition of Full Dataset', fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from statsmodels.robust import mad

# --- Load & preprocess ---
df = df.sort_index()
df = df.asfreq('1T')  # 1-min frequency
df['value'] = df['value'].interpolate(method='time')

# --- Extract last 1-hour window first ---
now = df.index.max()
last_hour_start = now - pd.Timedelta(hours=1)

# Split data: historical for pattern learning, recent for anomaly detection
df_historic = df.loc[df.index < last_hour_start].copy()
df_recent = df.loc[df.index >= last_hour_start].copy()

# --- Quick STL on recent data only (much faster) ---
if len(df_recent) >= 60:  # Need minimum points for STL
    period = min(60, len(df_recent) // 2)  # Adaptive period for short sequences
    stl_recent = STL(df_recent['value'], period=period, seasonal=7, trend=None, robust=True)
    res_recent = stl_recent.fit()
    resid_recent = res_recent.resid
else:
    # Fallback: simple detrending for very short sequences
    resid_recent = df_recent['value'] - df_recent['value'].rolling(window=5, center=True).mean()

# --- Vectorized pattern analysis ---
df_historic['minute'] = df_historic.index.hour * 60 + df_historic.index.minute
df_recent['minute'] = df_recent.index.hour * 60 + df_recent.index.minute

# Precompute historical stats (vectorized)
stats = df_historic.groupby('minute')['value'].agg(['median', mad]).rename(columns={'mad': 'mad_val'})

# --- Vectorized anomaly detection ---
# Global threshold from recent residuals
global_thresh = 3 * mad(resid_recent.dropna())

# Merge recent data with historical stats
df_recent_merged = df_recent.merge(stats, left_on='minute', right_index=True, how='left')

# Fill missing stats with global median/mad for minutes not in historical data
df_recent_merged['median'].fillna(df_historic['value'].median(), inplace=True)
df_recent_merged['mad_val'].fillna(mad(df_historic['value']), inplace=True)

# Vectorized anomaly conditions
resid_condition = np.abs(resid_recent) > global_thresh
pattern_condition = np.abs(df_recent_merged['value'] - df_recent_merged['median']) > 2.5 * df_recent_merged['mad_val']

# Combine conditions
anomaly_mask = resid_condition & pattern_condition

# Extract anomalies
anomaly_timestamps = df_recent.index[anomaly_mask]
anomaly_values = df_recent['value'][anomaly_mask]

# --- Create output DataFrame ---
if len(anomaly_timestamps) > 0:
    anomaly_df = pd.DataFrame({
        'timestamp': anomaly_timestamps,
        'value': anomaly_values
    })
    anomaly_df['clock'] = anomaly_df['timestamp'].astype(np.int64) // 10**9
    anomaly_df = anomaly_df[['clock', 'value']]
else:
    anomaly_df = pd.DataFrame(columns=['clock', 'value'])

print(f"\nDetected Anomalies (Last 1 Hour, Optimized): {len(anomaly_df)} anomalies")
print(anomaly_df)

# --- Optimized Plot ---
plt.figure(figsize=(14, 6))

# Plot only recent data for speed (or sample historical data)
recent_plot_data = df.tail(2880) if len(df) > 2880 else df  # Last 2 days max
plt.plot(recent_plot_data.index, recent_plot_data['value'], label='Data', color='#36A2EB', alpha=0.7)

if len(anomaly_df) > 0:
    plt.scatter(
        pd.to_datetime(anomaly_df['clock'], unit='s'),
        anomaly_df['value'], 
        color='#FF6384', 
        label=f'Anomalies ({len(anomaly_df)})',
        s=50, 
        zorder=5
    )

plt.axvspan(last_hour_start, now, color='gray', alpha=0.1, label='Detection Window')
plt.title('Fast Anomaly Detection with Pattern Suppression')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.tight_layout()
plt.show()

# --- Performance info ---
print(f"\nProcessing summary:")
print(f"Historical data points: {len(df_historic):,}")
print(f"Recent data points: {len(df_recent):,}")
print(f"Unique minute patterns: {len(stats):,}")

In [ ]:
df=data1
df.head()
df.size

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from statsmodels.robust import mad

# --- Load & preprocess ---
df = df.sort_index()
print(df)

In [ ]:
df['clock'] = pd.to_datetime(df['clock'], unit='s')
df

In [ ]:
df = df.set_index('clock')

In [ ]:

if not isinstance(df.index, pd.DatetimeIndex):
    df = df.set_index('clock')
df = df.asfreq('1min')  # 1-min frequency
df

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from statsmodels.robust import mad
import warnings
warnings.filterwarnings('ignore')

def ensure_odd_and_valid(value, minimum=3):
    """Ensure value is odd and >= minimum"""
    value = max(minimum, int(value))
    return value if value % 2 == 1 else value + 1

def calculate_stl_parameters(data_length, base_period=1440):
    """Calculate valid STL parameters based on data length"""
    # Ensure we have enough data for meaningful decomposition
    if data_length < 60:
        return None, None, None
    
    # Period: Try to use daily pattern (1440 min), but adapt for short data
    period = min(base_period, max(2, data_length // 4))
    
    # Seasonal: Should be odd and >= 3, typically period/4 to period/2
    seasonal_base = max(3, period // 4)
    seasonal = ensure_odd_and_valid(seasonal_base, 3)
    
    # Trend: Must be odd, >= 3, and > period
    # Rule of thumb: 1.5 * period, but ensure it's not too large
    trend_base = max(period + 2, int(1.5 * period))
    trend_base = min(trend_base, data_length // 2)  # Don't exceed half the data length
    trend = ensure_odd_and_valid(trend_base, period + 2)
    
    # Final validation
    if trend <= period:
        trend = ensure_odd_and_valid(period + 2, period + 2)
    
    # If trend is still too large, reduce period
    if trend >= data_length // 2:
        period = max(2, data_length // 6)
        trend = ensure_odd_and_valid(period + 2, period + 2)
        seasonal = ensure_odd_and_valid(max(3, period // 4), 3)
    
    return period, seasonal, trend

def chunked_stl_decomposition(data, chunk_size_hours=24, overlap_hours=2):
    """
    Process STL in overlapping chunks with improved parameter handling
    """
    chunk_size = chunk_size_hours * 60  # Convert to minutes
    overlap_size = overlap_hours * 60
    residuals = pd.Series(index=data.index, dtype=float)
    
    start_idx = 0
    total_chunks = (len(data) + chunk_size - 1) // chunk_size
    
    print(f"Processing {total_chunks} chunks...")
    
    for chunk_num in range(total_chunks):
        print(f"Processing chunk {chunk_num + 1}/{total_chunks}...")
        
        # Define chunk boundaries
        end_idx = min(start_idx + chunk_size + overlap_size, len(data))
        chunk_data = data.iloc[start_idx:end_idx]
        
        if len(chunk_data) < 60:  # Skip if too small for meaningful STL
            print(f"  Chunk {chunk_num + 1} too small ({len(chunk_data)} points), using simple detrending...")
            rolling_mean = chunk_data.rolling(window=min(10, len(chunk_data)//2), center=True).mean()
            chunk_residuals = chunk_data - rolling_mean
            
            if chunk_num == 0:
                residuals.iloc[start_idx:end_idx] = chunk_residuals
            else:
                skip_start = start_idx + overlap_size
                residuals.iloc[skip_start:end_idx] = chunk_residuals.iloc[overlap_size:]
            
            start_idx = end_idx - overlap_size
            continue
        
        # Calculate valid STL parameters
        period, seasonal, trend = calculate_stl_parameters(len(chunk_data))
        
        if period is None:
            print(f"  Chunk {chunk_num + 1} using simple detrending (insufficient data)...")
            rolling_mean = chunk_data.rolling(window=min(20, len(chunk_data)//3), center=True).mean()
            chunk_residuals = chunk_data - rolling_mean
        else:
            print(f"  STL params - Period: {period}, Seasonal: {seasonal}, Trend: {trend}")
            
            try:
                stl = STL(chunk_data, period=period, seasonal=seasonal, trend=trend, robust=True)
                res = stl.fit()
                chunk_residuals = res.resid
                print(f"  STL decomposition successful")
                
            except Exception as e:
                print(f"  STL failed for chunk {chunk_num + 1}, using simple detrending: {e}")
                # Fallback: simple detrending
                rolling_mean = chunk_data.rolling(window=min(60, len(chunk_data)//4), center=True).mean()
                chunk_residuals = chunk_data - rolling_mean
        
        # Store residuals (avoid overlap except for first chunk)
        if chunk_num == 0:
            residuals.iloc[start_idx:end_idx] = chunk_residuals
        else:
            skip_start = start_idx + overlap_size
            residuals.iloc[skip_start:end_idx] = chunk_residuals.iloc[overlap_size:]
        
        # Move to next chunk
        start_idx = end_idx - overlap_size
    
    return residuals

def compute_adaptive_stats(data, window_days=30):
    """Compute adaptive statistics using rolling windows"""
    window_size = window_days * 1440  # Convert days to minutes
    
    stats_list = []
    for minute in range(1440):  # All minutes in a day
        minute_data = data[data['minute'] == minute]['value']
        
        if len(minute_data) == 0:
            continue
            
        # Use rolling statistics if enough data
        if len(minute_data) > window_size:
            rolling_median = minute_data.rolling(window=window_size, min_periods=window_size//2).median()
            rolling_mad = minute_data.rolling(window=window_size, min_periods=window_size//2).apply(lambda x: mad(x) if len(x) > 0 else 0)
            
            # Take the last (most recent) values
            median_val = rolling_median.iloc[-1] if not rolling_median.empty else minute_data.median()
            mad_val = rolling_mad.iloc[-1] if not rolling_mad.empty else mad(minute_data)
        else:
            # Use all historical data if not enough for rolling window
            median_val = minute_data.median()
            mad_val = mad(minute_data) if len(minute_data) > 1 else minute_data.std()
            
        stats_list.append({'minute': minute, 'median': median_val, 'mad_val': mad_val})
    
    return pd.DataFrame(stats_list).set_index('minute')

def detect_anomalies(df):
    """
    Main anomaly detection function with improved robustness
    """
    print(f"Processing {len(df):,} data points...")
    
    # Ensure data is sorted by time
    df = df.sort_index()
    
    # Interpolate missing values
    df['value'] = df['value'].interpolate(method='time')
    
    # --- STL decomposition ---
    if len(df) > 10080:  # More than 1 week of data
        print("Large dataset detected, using chunked processing...")
        residuals = chunked_stl_decomposition(df['value'])
    else:
        print("Small dataset, using full STL...")
        period, seasonal, trend = calculate_stl_parameters(len(df))
        
        if period is not None:
            print(f"STL params - Period: {period}, Seasonal: {seasonal}, Trend: {trend}")
            try:
                stl = STL(df['value'], period=period, seasonal=seasonal, trend=trend, robust=True)
                res = stl.fit()
                residuals = res.resid
                print("STL decomposition successful")
            except Exception as e:
                print(f"STL failed, using simple detrending: {e}")
                rolling_mean = df['value'].rolling(window=min(1440, len(df)//4), center=True).mean()
                residuals = df['value'] - rolling_mean
        else:
            print("Dataset too small for STL, using simple detrending...")
            rolling_mean = df['value'].rolling(window=min(60, len(df)//4), center=True).mean()
            residuals = df['value'] - rolling_mean
    
    # --- Pattern analysis ---
    print("Computing historical patterns...")
    df['minute'] = df.index.hour * 60 + df.index.minute
    
    # Compute adaptive statistics
    stats = compute_adaptive_stats(df)
    
    # --- Anomaly detection ---
    print("Detecting anomalies...")
    
    # Global threshold
    global_thresh = 3 * mad(residuals.dropna()) if len(residuals.dropna()) > 0 else 3 * df['value'].std()
    print(f"Global residual threshold: {global_thresh:.3f}")
    
    # Merge with historical stats
    df_merged = df.merge(stats, left_on='minute', right_index=True, how='left')
    
    # Fill missing stats with global values
    global_median = df['value'].median()
    global_mad = mad(df['value']) if len(df) > 1 else df['value'].std()
    df_merged['median'].fillna(global_median, inplace=True)
    df_merged['mad_val'].fillna(global_mad, inplace=True)
    
    # Vectorized anomaly conditions
    resid_condition = np.abs(residuals) > global_thresh
    pattern_condition = np.abs(df_merged['value'] - df_merged['median']) > 2.5 * df_merged['mad_val']
    
    # Additional condition: significant deviation from recent trend
    recent_trend = df['value'].rolling(window=min(60, len(df)//10), center=True).mean()
    trend_std = df['value'].rolling(window=min(1440, len(df)//2)).std()
    trend_condition = np.abs(df['value'] - recent_trend) > 3 * trend_std
    
    # Combine all conditions (any two must be true for robustness)
    condition_sum = (resid_condition.astype(int) + 
                    pattern_condition.astype(int) + 
                    trend_condition.astype(int))
    anomaly_mask = condition_sum >= 2
    
    # Extract anomalies
    anomaly_timestamps = df.index[anomaly_mask]
    anomaly_values = df['value'][anomaly_mask]
    
    # Create output DataFrame
    if len(anomaly_timestamps) > 0:
        anomaly_df = pd.DataFrame({
            'timestamp': anomaly_timestamps,
            'value': anomaly_values
        })
        anomaly_df['clock'] = anomaly_df['timestamp'].astype(np.int64) // 10**9
        anomaly_df = anomaly_df[['clock', 'value']]
    else:
        anomaly_df = pd.DataFrame(columns=['clock', 'value'])
    
    print(f"\nDetected Anomalies: {len(anomaly_df):,}")
    print(f"Anomaly rate: {len(anomaly_df)/len(df)*100:.2f}%")
    
    if len(anomaly_df) > 0:
        print("\nFirst 10 anomalies:")
        print(anomaly_df.head(10))
    
    # --- Visualization ---
    create_anomaly_plots(df, anomaly_df, residuals, global_thresh)
    
    return anomaly_df, residuals

def create_anomaly_plots(df, anomaly_df, residuals, global_thresh):
    """Create comprehensive anomaly detection plots"""
    print("Creating visualization...")
    
    plt.figure(figsize=(16, 12))
    
    # Subplot 1: Full data with anomalies
    plt.subplot(4, 1, 1)
    # Sample data for plotting if too large
    plot_data = df.iloc[::max(1, len(df)//5000)] if len(df) > 5000 else df
    plt.plot(plot_data.index, plot_data['value'], label='Data', color='#36A2EB', alpha=0.6, linewidth=0.8)
    
    if len(anomaly_df) > 0:
        # Sample anomalies for plotting if too many
        plot_anomalies = anomaly_df.iloc[::max(1, len(anomaly_df)//1000)] if len(anomaly_df) > 1000 else anomaly_df
        plt.scatter(
            pd.to_datetime(plot_anomalies['clock'], unit='s'),
            plot_anomalies['value'], 
            color='#FF6384', 
            label=f'Anomalies ({len(anomaly_df):,})',
            s=15, 
            alpha=0.8,
            zorder=5
        )
    
    plt.title('Anomaly Detection Results - Full Dataset')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Subplot 2: Residuals
    plt.subplot(4, 1, 2)
    plot_residuals = residuals.iloc[::max(1, len(residuals)//5000)] if len(residuals) > 5000 else residuals
    plt.plot(plot_residuals.index, plot_residuals.values, color='orange', alpha=0.7, linewidth=0.6)
    plt.axhline(y=global_thresh, color='red', linestyle='--', alpha=0.7, label=f'Threshold (±{global_thresh:.2f})')
    plt.axhline(y=-global_thresh, color='red', linestyle='--', alpha=0.7)
    plt.title('STL Residuals')
    plt.ylabel('Residual')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Subplot 3: Anomaly density over time
    plt.subplot(4, 1, 3)
    if len(anomaly_df) > 0:
        anomaly_times = pd.to_datetime(anomaly_df['clock'], unit='s')
        # Create hourly bins for anomaly density
        hourly_counts = anomaly_times.dt.floor('H').value_counts().sort_index()
        plt.plot(hourly_counts.index, hourly_counts.values, color='red', marker='o', markersize=3, linewidth=1)
        plt.title('Anomaly Density Over Time (Hourly)')
        plt.ylabel('Anomaly Count')
        plt.grid(True, alpha=0.3)
    else:
        plt.text(0.5, 0.5, 'No anomalies detected', ha='center', va='center', 
                transform=plt.gca().transAxes, fontsize=12)
        plt.title('Anomaly Density Over Time')
    
    # Subplot 4: Statistics summary
    plt.subplot(4, 1, 4)
    if len(anomaly_df) > 0:
        # Value distribution
        plt.hist(df['value'], bins=50, alpha=0.7, color='blue', label='Normal Data', density=True)
        plt.hist(anomaly_df['value'], bins=20, alpha=0.8, color='red', label='Anomalies', density=True)
        plt.title('Value Distribution Comparison')
        plt.xlabel('Value')
        plt.ylabel('Density')
        plt.legend()
        plt.grid(True, alpha=0.3)
    else:
        plt.hist(df['value'], bins=50, alpha=0.7, color='blue', label='Data Distribution')
        plt.title('Data Value Distribution')
        plt.xlabel('Value')
        plt.ylabel('Frequency')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print(f"\n{'='*60}")
    print("ANOMALY DETECTION SUMMARY")
    print(f"{'='*60}")
    print(f"Total data points processed: {len(df):,}")
    print(f"Total anomalies detected: {len(anomaly_df):,}")
    print(f"Anomaly rate: {len(anomaly_df)/len(df)*100:.3f}%")
    print(f"Global residual threshold: {global_thresh:.3f}")
    print(f"Data time range: {df.index.min()} to {df.index.max()}")
    print(f"Processing completed successfully!")

# Example usage:
# Assuming your DataFrame 'df' is already loaded with 'clock' and 'value' columns
anomaly_df, residuals = detect_anomalies(df)

In [ ]:
data = pd.read_csv(
	'https://raw.githubusercontent.com/numenta'
	'/NAB/master/data/realKnownCause/ambient'
	'_temperature_system_failure.csv')
data = data.rename(columns={'timestamp': 'clock'})
data
df = data
df


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from statsmodels.robust import mad
from joblib import Parallel, delayed, Memory
import warnings
warnings.filterwarnings('ignore')

# Cache setup for intermediate results
memory = Memory("cache_dir", verbose=0)

def ensure_odd_and_valid(value, minimum=3):
    """Ensure value is odd and >= minimum"""
    value = max(minimum, int(value))
    return value if value % 2 == 1 else value + 1

def calculate_stl_parameters(data_length, base_period=288):  # Adjusted for 5-min intervals
    """Calculate valid STL parameters based on data length"""
    if data_length < 60:
        return None, None, None
    period = min(base_period, max(2, data_length // 4))
    seasonal = ensure_odd_and_valid(period // 8, 3)  # Smaller seasonal window
    trend = ensure_odd_and_valid(period + 2, period + 2)
    if trend >= data_length // 2:
        period = max(2, data_length // 6)
        trend = ensure_odd_and_valid(period + 2, period + 2)
        seasonal = ensure_odd_and_valid(max(3, period // 4), 3)
    return period, seasonal, trend

def process_chunk(chunk_data, start_idx, end_idx, overlap_size, chunk_num, total_chunks):
    """Process a single chunk for STL decomposition"""
    print(f"Processing chunk {chunk_num + 1}/{total_chunks}...")
    if len(chunk_data) < 60:
        print(f"  Chunk {chunk_num + 1} too small, using simple detrending...")
        rolling_mean = chunk_data.rolling(window=min(10, len(chunk_data)//2), center=True).mean()
        chunk_residuals = chunk_data - rolling_mean
    else:
        period, seasonal, trend = calculate_stl_parameters(len(chunk_data))
        if period is None:
            print(f"  Chunk {chunk_num + 1} using simple detrending...")
            rolling_mean = chunk_data.rolling(window=min(20, len(chunk_data)//3), center=True).mean()
            chunk_residuals = chunk_data - rolling_mean
        else:
            print(f"  STL params - Period: {period}, Seasonal: {seasonal}, Trend: {trend}")
            robust = len(chunk_data) > 1000  # Only use robust for larger chunks
            try:
                stl = STL(chunk_data, period=period, seasonal=seasonal, trend=trend, robust=robust)
                res = stl.fit()
                chunk_residuals = res.resid
                print(f"  STL decomposition successful")
            except Exception as e:
                print(f"  STL failed for chunk {chunk_num + 1}: {e}")
                rolling_mean = chunk_data.rolling(window=min(60, len(chunk_data)//4), center=True).mean()
                chunk_residuals = chunk_data - rolling_mean
    return chunk_num, start_idx, end_idx, overlap_size, chunk_residuals

@memory.cache
def chunked_stl_decomposition(data, chunk_size_hours=24, overlap_hours=2):
    """Process STL in overlapping chunks with parallelization"""
    chunk_size = chunk_size_hours * 60 // 5  # Adjusted for 5-min intervals
    overlap_size = overlap_hours * 60 // 5
    residuals = pd.Series(index=data.index, dtype=float)
    total_chunks = (len(data) + chunk_size - 1) // chunk_size

    print(f"Processing {total_chunks} chunks...")
    chunks = [
        (data.iloc[i:i + chunk_size + overlap_size], i, min(i + chunk_size + overlap_size, len(data)), overlap_size, chunk_num, total_chunks)
        for chunk_num, i in enumerate(range(0, len(data), chunk_size))
    ]

    results = Parallel(n_jobs=-1)(delayed(process_chunk)(*chunk) for chunk in chunks)

    for chunk_num, start_idx, end_idx, overlap_size, chunk_residuals in results:
        if chunk_num == 0:
            residuals.iloc[start_idx:end_idx] = chunk_residuals
        else:
            skip_start = start_idx + overlap_size
            residuals.iloc[skip_start:end_idx] = chunk_residuals.iloc[overlap_size:]

    return residuals

def compute_adaptive_stats(data, window_days=30):
    """Compute adaptive statistics using grouped operations"""
    window_size = window_days * 1440 // 5  # Adjusted for 5-min intervals
    stats_list = []
    grouped = data.groupby('minute')['value']
    
    for minute, group_data in grouped:
        if len(group_data) == 0:
            continue
        if len(group_data) > window_size:
            rolling_median = group_data.rolling(window=window_size, min_periods=window_size//2).median().iloc[-1]
            rolling_mad = group_data.rolling(window=window_size, min_periods=window_size//2).apply(lambda x: mad(x) if len(x) > 0 else 0).iloc[-1]
        else:
            rolling_median = group_data.median()
            rolling_mad = mad(group_data) if len(group_data) > 1 else group_data.std()
        stats_list.append({'minute': minute, 'median': rolling_median, 'mad_val': rolling_mad})
    
    return pd.DataFrame(stats_list).set_index('minute')

def create_anomaly_plots(df, anomaly_df, residuals, global_thresh):
    """Create comprehensive anomaly detection plots"""
    print("Creating visualization...")
    plt.figure(figsize=(16, 12))

    # Downsample for plotting
    sample_rate = max(1, len(df) // 5000)
    plot_data = df.iloc[::sample_rate]
    plot_residuals = residuals.iloc[::sample_rate]

    # Subplot 1: Full data with anomalies
    plt.subplot(4, 1, 1)
    plt.plot(plot_data.index, plot_data['value'], label='Data', color='#36A2EB', alpha=0.6, linewidth=0.8)
    if len(anomaly_df) > 0:
        plot_anomalies = anomaly_df.iloc[::max(1, len(anomaly_df)//1000)]
        plt.scatter(
            pd.to_datetime(plot_anomalies['clock'], unit='s'),
            plot_anomalies['value'], 
            color='#FF6384', 
            label=f'Anomalies ({len(anomaly_df):,})',
            s=15, 
            alpha=0.8,
            zorder=5
        )
    plt.title('Anomaly Detection Results - Full Dataset')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Subplot 2: Residuals
    plt.subplot(4, 1, 2)
    plt.plot(plot_residuals.index, plot_residuals.values, color='orange', alpha=0.7, linewidth=0.6)
    plt.axhline(y=global_thresh, color='red', linestyle='--', alpha=0.7, label=f'Threshold (±{global_thresh:.2f})')
    plt.axhline(y=-global_thresh, color='red', linestyle='--', alpha=0.7)
    plt.title('STL Residuals')
    plt.ylabel('Residual')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Subplot 3: Anomaly density over time
    plt.subplot(4, 1, 3)
    if len(anomaly_df) > 0:
        anomaly_times = pd.to_datetime(anomaly_df['clock'], unit='s')
        hourly_counts = anomaly_times.dt.floor('H').value_counts().sort_index()
        plt.plot(hourly_counts.index, hourly_counts.values, color='red', marker='o', markersize=3, linewidth=1)
        plt.title('Anomaly Density Over Time (Hourly)')
        plt.ylabel('Anomaly Count')
        plt.grid(True, alpha=0.3)
    else:
        plt.text(0.5, 0.5, 'No anomalies detected', ha='center', va='center', 
                transform=plt.gca().transAxes, fontsize=12)
        plt.title('Anomaly Density Over Time')

    # Subplot 4: Statistics summary
    plt.subplot(4, 1, 4)
    plt.hist(plot_data['value'], bins=30, alpha=0.7, color='blue', label='Normal Data', density=True)
    if len(anomaly_df) > 0:
        plt.hist(anomaly_df['value'], bins=10, alpha=0.8, color='red', label='Anomalies', density=True)
    plt.title('Value Distribution Comparison')
    plt.xlabel('Value')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

def detect_anomalies(df):
    """Main anomaly detection function with improved robustness and speed"""
    print(f"Processing {len(df):,} data points...")
    
    # Ensure data is sorted by time
    df = df.sort_index()
    
    # Add minute column to original df
    df['minute'] = df.index.hour * 60 + df.index.minute
    
    # Downsample to 5-minute intervals
    df_downsampled = df.resample('5min').mean().interpolate(method='time')
    df_downsampled['minute'] = df_downsampled.index.hour * 60 + df_downsampled.index.minute
    
    # STL decomposition
    if len(df_downsampled) > 2016:  # 1 week at 5-min intervals
        print("Large dataset detected, using chunked processing...")
        residuals = chunked_stl_decomposition(df_downsampled['value'])
    else:
        print("Small dataset, using full STL...")
        period, seasonal, trend = calculate_stl_parameters(len(df_downsampled))
        if period is not None:
            print(f"STL params - Period: {period}, Seasonal: {seasonal}, Trend: {trend}")
            robust = len(df_downsampled) > 1000
            try:
                stl = STL(df_downsampled['value'], period=period, seasonal=seasonal, trend=trend, robust=robust)
                res = stl.fit()
                residuals = res.resid
                print("STL decomposition successful")
            except Exception as e:
                print(f"STL failed: {e}")
                rolling_mean = df_downsampled['value'].rolling(window=min(288, len(df_downsampled)//4), center=True).mean()
                residuals = df_downsampled['value'] - rolling_mean
        else:
            print("Dataset too small for STL, using simple detrending...")
            rolling_mean = df_downsampled['value'].rolling(window=min(60, len(df_downsampled)//4), center=True).mean()
            residuals = df_downsampled['value'] - rolling_mean
    
    # Interpolate residuals back to original index
    residuals = residuals.reindex(df.index, method='nearest').interpolate(method='time')
    
    # Compute adaptive statistics
    print("Computing historical patterns...")
    stats = compute_adaptive_stats(df_downsampled)
    
    # Anomaly detection
    print("Detecting anomalies...")
    global_thresh = 5 * mad(residuals.dropna()) if len(residuals.dropna()) > 0 else 3 * df['value'].std()
    print(f"Global residual threshold: {global_thresh:.3f}")
    
    df_merged = df.merge(stats, left_on='minute', right_index=True, how='left')
    df_merged['median'].fillna(df['value'].median(), inplace=True)
    df_merged['mad_val'].fillna(mad(df['value']) if len(df) > 1 else df['value'].std(), inplace=True)
    
    resid_condition = np.abs(residuals) > global_thresh
    pattern_condition = np.abs(df_merged['value'] - df_merged['median']) > 2.5 * df_merged['mad_val']
    recent_trend = df['value'].rolling(window=min(60, len(df)//10), center=True).mean()
    trend_std = df['value'].rolling(window=min(1440, len(df)//2)).std()
    trend_condition = np.abs(df['value'] - recent_trend) > 3 * trend_std
    
    condition_sum = (resid_condition.astype(int) + 
                    pattern_condition.astype(int) + 
                    trend_condition.astype(int))
    anomaly_mask = condition_sum >= 2
    
    anomaly_df = pd.DataFrame({
        'timestamp': df.index[anomaly_mask],
        'value': df['value'][anomaly_mask]
    })
    anomaly_df['clock'] = anomaly_df['timestamp'].astype(np.int64) // 10**9
    anomaly_df = anomaly_df[['clock', 'value']]
    
    print(f"\nDetected Anomalies: {len(anomaly_df):,}")
    print(f"Anomaly rate: {len(anomaly_df)/len(df)*100:.2f}%")
    
    if len(anomaly_df) > 0:
        print("\nFirst 10 anomalies:")
        print(anomaly_df.head(10))
    
    # Create visualization (comment out if not needed)
    create_anomaly_plots(df, anomaly_df, residuals, global_thresh)
    
    print(f"\n{'='*60}")
    print("ANOMALY DETECTION SUMMARY")
    print(f"{'='*60}")
    print(f"Total data points processed: {len(df):,}")
    print(f"Total anomalies detected: {len(anomaly_df):,}")
    print(f"Anomaly rate: {len(anomaly_df)/len(df)*100:.3f}%")
    print(f"Global residual threshold: {global_thresh:.3f}")
    print(f"Data time range: {df.index.min()} to {df.index.max()}")
    print(f"Processing completed successfully!")
    
    return anomaly_df, residuals

# Example usage:
# df = pd.DataFrame({'value': values, 'timestamp': timestamps})
# df.index = pd.to_datetime(df['timestamp'])
anomaly_df, residuals = detect_anomalies(df)